# Smart NIC with DPDK, VPP and Pktgen-DPDK

DPDK is the Data Plane Development Kit that consists of libraries to accelerate packet processing workloads running on a wide variety of CPU architectures. It is designed to run on x86, POWER, and ARM processors and licensed under the Open-Source BSD License.

VPP is a high performance network stack that provides different dataplanes (Linux, RDMA and DPDK) and can be used as vSwitches, vRouters, Gateways, Firewalls and Load-Balancers.
Pktgen-DPDK is a reference traffic generator, capable of generating 100Gbps using a single CPU core

This notebook depicts how to attach Smart NICs to DPDK and send traffic to the DPDK connected NICs via TRex.

In [1]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,f1f5b0f8-0eef-40f3-940e-c4c8ff2d60ca
Bastion Host,bastion.fabric-testbed.net
Bastion Username,marcos_schwarz_0034669026
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Sites to avoid,


## Create the Experiment Slice

The following creates two nodes with NIC_ConnectX_6/NIC_ConnectX_5 NICs connected to an isolated local Ethernet. 
- `vpp-node`: with one port connected to local L2 network. We will run VPP application here
- `pktgen-node`: with one port connected to local L2 network. We wull run DPDK Packet Generator her 

In addition to local L2 network, each node is connected to FABNETv4 to be able to download the drivers from the Storage Node.

<img src="./images/dpdk-pktgen-smart-nic.png"><br>

In [2]:
slice_name = 'MySlice-dpdk'
site = fablib.get_random_site(filter_function=lambda x: x['nic_connectx_6_available'])
print(f"Site: {site}")

vpp_node_name = 'vpp-node'
pktgen_node_name = 'pktgen-node'

image_name = "docker_ubuntu_22"

Site: AMST


In [3]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Network
net1 = slice.add_l2network(name='net1', subnet=IPv4Network("192.168.0.0/24"))
net2 = slice.add_l2network(name='net2', subnet=IPv4Network("192.168.1.0/24"))

# pktgen_node
pktgen_node = slice.add_node(name=pktgen_node_name, site=site, cores=8, ram=16, disk=20, image=image_name)
pktgen_node.add_fabnet()
pktgen_node_iface1, pktgen_node_iface2 = pktgen_node.add_component(model='NIC_ConnectX_6', name='nic1').get_interfaces()

# vpp_node
vpp_node = slice.add_node(name=vpp_node_name, site=site, cores=8, ram=16, disk=20, image=image_name)
vpp_node.add_fabnet()
vpp_node_iface1, vpp_node_iface2 = vpp_node.add_component(model='NIC_ConnectX_6', name='nic1').get_interfaces()

pktgen_node_iface1.set_mode('auto')
pktgen_node_iface2.set_mode('auto')
vpp_node_iface1.set_mode('auto')
vpp_node_iface2.set_mode('auto')

net1.add_interface(pktgen_node_iface1)
net1.add_interface(vpp_node_iface1)

net2.add_interface(pktgen_node_iface2)
net2.add_interface(vpp_node_iface2)

pktgen_node.add_post_boot_upload_directory('pktgen-dpdk','.')
vpp_node.add_post_boot_upload_directory('vpp','.')

for n in slice.get_nodes():
    n.add_post_boot_upload_directory('node_tools','.')
    n.add_post_boot_execute('chmod +x node_tools/*')
    n.add_post_boot_execute('sudo node_tools/enable_docker.sh {{ _self_.image }} ')

#Submit Slice Request
slice.submit();


Retry: 29, Time: 749 sec


ID,cc138509-0880-4028-b494-303ca616414a
Name,MySlice-dpdk
Lease Expiration (UTC),2024-04-26 14:39:37 +0000
Lease Start (UTC),2024-04-25 14:39:38 +0000
Project ID,f1f5b0f8-0eef-40f3-940e-c4c8ff2d60ca
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
8d55177f-be3c-4575-aa3d-9ba9ffbcc00c,pktgen-node,8,16,100,docker_ubuntu_22,qcow2,amst-w2.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fef3:b197,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fef3:b197,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
521d241a-de28-4ee0-b98f-df5ff95110eb,vpp-node,8,16,100,docker_ubuntu_22,qcow2,amst-w2.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fe7a:6a9a,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fe7a:6a9a,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
111d7c94-c3cf-486d-95ec-79ff22c5826c,FABNET_IPv4_AMST,L3,FABNetv4,AMST,10.145.20.0/24,10.145.20.1,Active,
d4247242-7ca9-44f3-8400-05f4c3dffce9,net1,L2,L2Bridge,AMST,192.168.0.0/24,None,Active,
c21f341c-3b96-4d64-9897-332131623f66,net2,L2,L2Bridge,AMST,192.168.1.0/24,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
pktgen-node-nic1-p1,p1,pktgen-node,net1,100,auto,,10:70:FD:E5:E7:08,enp7s0np0,enp7s0np0,192.168.0.2,1
pktgen-node-nic1-p2,p2,pktgen-node,net2,100,auto,,10:70:FD:E5:E7:09,enp8s0np0,enp8s0np0,192.168.1.1,1
pktgen-node-FABNET_IPv4_AMST_nic-p1,p1,pktgen-node,FABNET_IPv4_AMST,100,auto,,02:DC:6F:4E:57:AF,enp9s0,enp9s0,10.145.20.2,4
vpp-node-nic1-p1,p1,vpp-node,net1,100,auto,,10:70:FD:E5:CD:D0,enp8s0np0,enp8s0np0,192.168.0.1,2
vpp-node-nic1-p2,p2,vpp-node,net2,100,auto,,10:70:FD:E5:CD:D1,enp9s0np0,enp9s0np0,192.168.1.2,2
vpp-node-FABNET_IPv4_AMST_nic-p1,p1,vpp-node,FABNET_IPv4_AMST,100,auto,,0E:EA:F8:31:B9:58,enp7s0,enp7s0,10.145.20.3,4



Time to print interfaces 749 seconds


## Verify connectivity between nodes on the local ethernet

In [8]:
vpp_node = slice.get_node(name=vpp_node_name)
pktgen_node = slice.get_node(name=pktgen_node_name)


pktgen_node_addr1 = pktgen_node.get_interface(network_name='net1').get_ip_addr()
pktgen_node_addr2 = pktgen_node.get_interface(network_name='net2').get_ip_addr()


stdout, stderr = vpp_node.execute(f'ping -c 5 {pktgen_node_addr1}')
stdout, stderr = vpp_node.execute(f'ping -c 5 {pktgen_node_addr2}')

PING 192.168.0.2 (192.168.0.2): 56 data bytes
64 bytes from 192.168.0.2: icmp_seq=0 ttl=64 time=0.304 ms
64 bytes from 192.168.0.2: icmp_seq=1 ttl=64 time=0.173 ms
64 bytes from 192.168.0.2: icmp_seq=2 ttl=64 time=0.204 ms
64 bytes from 192.168.0.2: icmp_seq=3 ttl=64 time=0.145 ms
64 bytes from 192.168.0.2: icmp_seq=4 ttl=64 time=0.175 ms
--- 192.168.0.2 ping statistics ---
5 packets transmitted, 5 packets received, 0% packet loss
round-trip min/avg/max/stddev = 0.145/0.200/0.304/0.055 ms
PING 192.168.1.1 (192.168.1.1): 56 data bytes
64 bytes from 192.168.1.1: icmp_seq=0 ttl=64 time=0.413 ms
64 bytes from 192.168.1.1: icmp_seq=1 ttl=64 time=0.143 ms
64 bytes from 192.168.1.1: icmp_seq=2 ttl=64 time=0.136 ms
64 bytes from 192.168.1.1: icmp_seq=3 ttl=64 time=0.148 ms
64 bytes from 192.168.1.1: icmp_seq=4 ttl=64 time=0.139 ms
--- 192.168.1.1 ping statistics ---
5 packets transmitted, 5 packets received, 0% packet loss
round-trip min/avg/max/stddev = 0.136/0.196/0.413/0.109 ms


## Start and configure VPP container

In [16]:
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose up -d')

 unknown shorthand flag: 'q' in -q
 

In [18]:
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose exec vpp vppctl create host-interface name {vpp_node_iface1.get_device_name()}')
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose exec vpp vppctl set int ip address host-{vpp_node_iface1.get_device_name()} 192.168.0.2/24')
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose exec vpp vppctl set interface state host-{vpp_node_iface1.get_device_name()} up')
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose exec vpp vppctl create host-interface name {vpp_node_iface2.get_device_name()}')
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose exec vpp vppctl set int ip address host-{vpp_node_iface2.get_device_name()} 192.168.1.2/24')
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose exec vpp vppctl set interface state host-{vpp_node_iface2.get_device_name()} up')
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose exec vpp vppctl show interface address')

host-enp8s0np0
host-enp9s0np0
host-enp8s0np0 (up):
  L3 192.168.0.2/24
host-enp9s0np0 (up):
  L3 192.168.1.2/24
local0 (dn):


## Configure MTU and firewall

In [15]:
pktgen_node = slice.get_node(name=pktgen_node_name)
vpp_node = slice.get_node(name=vpp_node_name)

pktgen_node_iface1 = pktgen_node.get_interface(network_name='net1')
pktgen_node_iface2 = pktgen_node.get_interface(network_name='net2')

vpp_node_iface1 = vpp_node.get_interface(network_name='net1')
vpp_node_iface2 = vpp_node.get_interface(network_name='net2')
stdout, stderr = vpp_node.execute(f'sudo ifconfig {vpp_node_iface1.get_device_name()} mtu 9000')
stdout, stderr = vpp_node.execute(f'sudo ifconfig {vpp_node_iface2.get_device_name()} mtu 9000')
stdout, stderr = vpp_node.execute(f'sudo nft insert rule ip filter FORWARD iifname {vpp_node_iface1.get_device_name()} oifname {vpp_node_iface2.get_device_name()} accept')
stdout, stderr = vpp_node.execute(f'sudo nft insert rule ip filter FORWARD iifname {vpp_node_iface2.get_device_name()} oifname {vpp_node_iface1.get_device_name()} accept')

## Pktgen-DPDK

In [21]:
stdout, stderr = pktgen_node.execute(f'echo 2048 | sudo tee /sys/kernel/mm/hugepages/hugepages-2048kB/nr_hugepages')
stdout, stderr = pktgen_node.execute(f'cd pktgen-dpdk; docker compose build -q')

2048


### Start Pktgen Application on pktgen-node

Open a SSH console for `pktgen-node` and start `pktgen` using the commands generated by the next cell:

In [22]:
print(f"Open a SSH console for {pktgen_node.get_name()} using the command: {pktgen_node.get_ssh_command()}")

Open a SSH console for pktgen-node using the command: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fef3:b197


In [23]:
print("Start DPDK PKTGEN using the command: ")
if "rocky" in pktgen_node.get_image():
    print("export PKG_CONFIG_PATH=/usr/local/lib64/pkgconfig")
    print("cd Pktgen-DPDK")
else:
    print("cd pktgen-dpdk")
print(f"docker compose run --rm pktgen-dpdk pktgen -a {pktgen_node_iface1.get_component().get_pci_addr()[0]} -a {pktgen_node_iface2.get_component().get_pci_addr()[1]}  -l 0-4 -n 3 -- -m \"[1:3].0, [2:4].1\" -j")

Start DPDK PKTGEN using the command: 
cd pktgen-dpdk
docker compose run --rm pktgen-dpdk pktgen -a 0000:07:00.0 -a 0000:08:00.0  -l 0-4 -n 3 -- -m "[1:3].0, [2:4].1" -j


After Pktgen is brought up, set up Pktgen and start traffic by running the commands generated by the next cell:

In [24]:
print(f"set 0 proto udp")
print("set 0 size 9000")
print(f"set 0 src ip {pktgen_node_iface1.get_ip_addr()}")
print(f"set 0 dst ip {pktgen_node_iface2.get_ip_addr()}")
print(f"set 0 dst mac {vpp_node_iface1.get_mac().lower()}")
print("start 0")
print("enable 1 process")

set 0 proto udp
set 0 size 9000
set 0 src ip 192.168.0.2
set 0 dst ip 192.168.1.1
set 0 dst mac 10:70:fd:e5:cd:d0
start 0
enable 1 process


Pktgen-DPDK will start updating the statistics on the top of the page, where you can spot the transmitted rate (TX) on port 0 and the receive rate on port 1 (RX) on line 6 (MBits/s Rx/Tx). In the following example it’s transmitting 98.4 Gbps (100753 Mbps / 1024) and receiving only 17 Gbps (17468 Mbps / 1024) using the Linux datapath.

Keep Pktgen running since it will be used to check the throughput of the SmartNIC using offloads using RDMA and DPDK.

## Setting up the VPP SmartNIC with RDMA offload
VPP provides a RDMA device driver[5], which uses RDMA APIs available on Nvidia ConnectX SmartNICS to offload Ethernet packets. To use it, the VPP container will be restarted to clean up the configurations and RDMA interfaces will be configured.

In [ ]:
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose restart')

  Container vpp-vpp-1  Restarting
 Container vpp-vpp-1  Started
 

In [ ]:
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose exec vpp vppctl create interface rdma host-if {vpp_node_iface1.get_device_name()} name rdma-0')
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose exec vpp vppctl set int ip address rdma-0 {vpp_node_iface1.get_ip_addr()}/24')
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose exec vpp vppctl set interface state rdma-0 up')
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose exec vpp vppctl create interface rdma host-if {vpp_node_iface2.get_device_name()} name rdma-1')
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose exec vpp vppctl set int ip address rdma-1 {vpp_node_iface2.get_ip_addr()}/24')
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose exec vpp vppctl set interface state rdma-1 up')
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose exec vpp vppctl show interface address')

local0 (dn):
rdma-0 (up):
  L3 192.168.0.1/24
rdma-1 (up):
  L3 192.168.1.2/24


Now go back to Pktgen-DPDK and verify that the receiving rate has increased by a factor of approximately 2.6 times, to 44 Gbps (45348 Mbps / 1024).

## Setting up the VPP SmartNIC with DPDK offload

In [35]:
stdout, stderr = vpp_node.execute(f'echo 2048 | sudo tee /proc/sys/vm/nr_hugepages')
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose down')
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose -f compose.yaml up -d')

2048
  Container vpp-vpp-1  Stopping
 Container vpp-vpp-1  Stopped
 Container vpp-vpp-1  Removing
 Container vpp-vpp-1  Removed
   Container vpp-vpp-1  Creating
 Container vpp-vpp-1  Created
 Container vpp-vpp-1  Starting
 Container vpp-vpp-1  Started
 

In [37]:
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose exec vpp vppctl set int ip address HundredGigabitEthernet8/0/0 {vpp_node_iface1.get_ip_addr()}/24')
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose exec vpp vppctl set interface state HundredGigabitEthernet8/0/0 up')
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose exec vpp vppctl set int ip address HundredGigabitEthernet9/0/0 {vpp_node_iface2.get_ip_addr()}/24')
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose exec vpp vppctl set interface state HundredGigabitEthernet9/0/0 up')
stdout, stderr = vpp_node.execute(f'cd vpp; docker compose exec vpp vppctl show interface address')

set interface ip address: failed to add 192.168.1.2/24 on HundredGigabitEthernet9/0/0 which conflicts with 192.168.1.2/24 for interface HundredGigabitEthernet9/0/0
HundredGigabitEthernet7/0/0/4096 (dn):
HundredGigabitEthernet8/0/0 (up):
  L3 192.168.0.1/24
HundredGigabitEthernet9/0/0 (up):
  L3 192.168.1.2/24
local0 (dn):


Finally, go back to Pktgen-DPDK and verify that the receiving rate has increased again by a factor of approximately 5.8 times compared to the original test, to 99.2 Gbps (101543 Mbps / 1024).

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice = fablib.get_slice(slice_name)
slice.delete()